# AEC Tech - IFC to Graph

This notebook was developed by David Andres Leon and Olga Poletkina based on the GraphML course of the Masters in Advanced Computation for Architecture and Design of IAAC
Macad 2025

# 1.Connect to google drive and install libraries

In [ ]:
# in case you need to unmount drive...
# drive.flush_and_unmount()


In [ ]:
# in case you need to access your drive...
# from google.colab import drive
# drive.mount("/content/drive", force_remount=True)

In [ ]:
%pip install ifcopenshell
%pip install networkx
%pip install plotly
%pip install seaborn

In [ ]:
import ifcopenshell
import networkx as nx
import plotly.graph_objects as go
import pandas as pd

: 

# 2. Extract data from IFC
## 2.1. Utility Function

### Introduction to IFC Structure (Industry Foundation Classes)

**IFC (Industry Foundation Classes)** is a standardized data model used to describe building and construction industry data. It enables interoperability between different software applications in Building Information Modeling (BIM).

At its core, an IFC file is made up of a large hierarchy of **entities**—objects like walls, doors, beams, or more abstract things like relationships and property sets. Each entity belongs to a specific **type**, e.g., `IfcWall`, `IfcDoor`, `IfcPropertySet`, and so on.

Key concepts relevant to the function:

- **IfcEntity:** A generic object in the IFC model. Each entity has basic properties such as GlobalId, Name, and Description.

- **IfcRelDefinesByProperties:** A relationship that links an entity to a set of properties or quantities.

- **IfcPropertySet:** A collection of properties (e.g., material, fire rating), where each property can be a single value (IfcPropertySingleValue).

- **IfcElementQuantity:** Stores measurable quantities (e.g., length, area, volume) associated with the entity.

- **wrappedValue:** A helper attribute used to access the actual value inside some IFC data types.

```
IFC Structure Overview (as used in extract_properties function)

            +------------------------+
            |      IfcEntity        |         (e.g., IfcWall, IfcDoor)
            +------------------------+
            | - GlobalId             |
            | - Name                 |
            | - Description          |
            | - ObjectType           |
            +------------------------+
                      |
                      | IsDefinedBy
                      v
        +-----------------------------+
        |  IfcRelDefinesByProperties |
        +-----------------------------+
                      |
                      | RelatingPropertyDefinition
        +-----------------------------+
        |       /             \       |
        v                             v
+------------------+      +------------------------+
|  IfcPropertySet  |      |  IfcElementQuantity    |
+------------------+      +------------------------+
| - HasProperties  |      | - Quantities           |
+------------------+      +------------------------+
        |                            |
        v                            v

+-------------------------+    +------------------------+
| IfcPropertySingleValue  |    |   Quantity (Length,    |
+-------------------------+    |    Area, Volume, ...)  |
| - Name                  |    +------------------------+
| - NominalValue          |    | - Name                 |
|   - wrappedValue        |    | - Value (wrappedValue) |
+-------------------------+    +------------------------+

```


In [ ]:
def extract_properties(entity):
    """Extracts general properties and quantities from an IFC entity."""
    data = {
        "GlobalId": entity.GlobalId,
        "Name": entity.Name,  # Initialize with entity.Name
        "Description": getattr(entity, "Description", None),
        "ObjectType": getattr(entity, "ObjectType", None),
        "IfcType": entity.is_a()
    }

    if data["Name"] is None and hasattr(entity, "IsDefinedBy"):
        for rel in entity.IsDefinedBy:
            if rel.is_a("IfcRelDefinesByProperties") and hasattr(rel, "RelatingPropertyDefinition"):
                prop_def = rel.RelatingPropertyDefinition
                if prop_def.is_a("IfcPropertySet"):
                    # Check if it's Pset_SpaceCommon
                    if prop_def.Name == "Pset_SpaceCommon":
                        for prop in prop_def.HasProperties:
                            if prop.is_a("IfcPropertySingleValue") and prop.Name == "Name":
                                data["Name"] = getattr(prop.NominalValue, "wrappedValue", None)
                                break  # Stop searching if found
                    else:  # Otherwise, try to find the name in other property sets
                        for prop in prop_def.HasProperties:
                            if prop.is_a("IfcPropertySingleValue"):
                                if prop.Name in ["Name", "name", "LongName", "longname"]:
                                    data["Name"] = getattr(prop.NominalValue, "wrappedValue", None)
                                    break  # Stop searching if found
                                property_value = getattr(prop.NominalValue, "wrappedValue", None)
                                if property_value in ["Name", "LongName"]:
                                    for inner_prop in prop_def.HasProperties:
                                        if inner_prop.is_a("IfcPropertySingleValue") and inner_prop.Name == property_value:
                                            data["Name"] = getattr(inner_prop.NominalValue, "wrappedValue", None)
                                            break
                                    break  # Stop searching if found
    # Check if Name is still None and LongName is available
    if data["Name"] is None and hasattr(entity, 'LongName'):
        data["Name"] = entity.LongName  # Assign LongName to Name if Name is None

    return data

todo -> Location parameter!



## 2.2. Load IFC File and Extract Data

In [ ]:
ifc_file = ifcopenshell.open('/content/racadvancedsampleproject.ifc')

rooms = ifc_file.by_type("IfcSpace")
space_boundaries = ifc_file.by_type("IfcRelSpaceBoundary")
# space_boundaries = ifc_file.by_type("IfcSpace")

In [ ]:
# Identify element types in space boundaries
element_types = set()
for rel in space_boundaries:
    if rel.RelatedBuildingElement:
        element_types.add(rel.RelatedBuildingElement.is_a())

print("Unique element types in space boundaries:")
for element_type in sorted(element_types):
    print("-", element_type)

Unique element types in space boundaries:
- IfcColumn
- IfcCurtainWall
- IfcDoor
- IfcSlab
- IfcWall
- IfcWallStandardCase
- IfcWindow


In [ ]:
# Define categories and colors
categories = {room.is_a(): "red" for room in rooms}

for rel in space_boundaries:
    if rel.RelatedBuildingElement:
        categories.setdefault(
            rel.RelatedBuildingElement.is_a(),
            f"#{hash(rel.RelatedBuildingElement.is_a()) & 0xFFFFFF:06x}"
        )

def get_node_color(ifc_type):
    """Return a color based on the IFC type or default to gray."""
    return categories.get(ifc_type, "gray")

In [ ]:
for i, room in enumerate(rooms):
    if i < 3:  # Only process the first three rooms
        room_properties = extract_properties(room)
        print(f"Room: {room_properties.get('Name', 'N/A')}")
        print("Properties:")
        for key, value in room_properties.items():
            print(f"  {key}: {value}")
        print("-" * 20)
    else:
        break  # Exit the loop after processing three rooms

Room: 101
Properties:
  GlobalId: 1GE3M13VHAWxhxwcp39Emh
  Name: 101
  Description: None
  ObjectType: None
  IfcType: IfcSpace
--------------------
Room: 102
Properties:
  GlobalId: 1GE3M13VHAWxhxwcp39EiJ
  Name: 102
  Description: None
  ObjectType: None
  IfcType: IfcSpace
--------------------
Room: 103
Properties:
  GlobalId: 1GE3M13VHAWxhxwcp39Eiz
  Name: 103
  Description: None
  ObjectType: None
  IfcType: IfcSpace
--------------------


In [ ]:
# Find and extract properties of a specific wall
wall_name_to_find = "LVL_2_1B_Wall_S48"
for wall in ifc_file.by_type("IfcWall"):
    wall_properties = extract_properties(wall)
    if wall_properties.get("Name") == wall_name_to_find:
        print(f"Wall: {wall_name_to_find}")

        # Print basic properties
        print("Properties:")
        for key, value in wall_properties.items():
            print(f"  {key}: {value}")

        # Extract and print custom properties from myPset
        print("\nCustom Properties (myPset):")
        custom_properties = {}  # Initialize an empty dict

        def get_custom_properties(entity):
            if hasattr(entity, "IsDefinedBy"):
                for definition in entity.IsDefinedBy:
                    if definition.is_a('IfcRelDefinesByProperties'):
                        property_set = definition.RelatingPropertyDefinition
                        if property_set.is_a('IfcPropertySet') and property_set.Name == "myPset":
                            for property in property_set.HasProperties:
                                if property.is_a('IfcPropertySingleValue'):
                                    # Automatically extract property name and value
                                    if property.NominalValue:
                                        if isinstance(property.NominalValue.wrappedValue, (list, tuple)):
                                            custom_properties[property.Name] = property.NominalValue.wrappedValue[0]
                                        else:
                                            custom_properties[property.Name] = property.NominalValue.wrappedValue
                                    else:
                                        custom_properties[property.Name] = None
            return custom_properties  # Return the dictionary

        custom_properties = get_custom_properties(wall)

        # Print all extracted custom properties
        for property_name, property_value in custom_properties.items():
            print(f"  {property_name}: {property_value}")

        break  # Stop searching once found

In [ ]:
rooms = ifc_file.by_type("IfcSpace")
print(f"Number of Rooms: {len(rooms)}")  # Added line

Number of Rooms: 116


In [ ]:
for room in rooms[:3]:  # Check the first 3 rooms
    print(f"Room: {room}")
    for attr in dir(room):  # Iterate through all attributes of the room object
        if not attr.startswith("_"):  # Exclude private attributes
            value = getattr(room, attr)
            print(f"  {attr}: {value}")

Room: #40710=IfcSpace('1GE3M13VHAWxhxwcp39Emh',#18,'101',$,$,#40664,#40709,'Vest.',.ELEMENT.,.INTERNAL.,$)
  BoundedBy: (#40744=IfcRelSpaceBoundary('1BlLKI1aU1Nh7m95veTSfz',#18,'2ndLevel',$,#40710,$,#40743,.VIRTUAL.,.INTERNAL.), #40755=IfcRelSpaceBoundary('0SuDgUeKTNJoQVzOIKy1GX',#18,'2ndLevel',$,#40710,$,#40754,.VIRTUAL.,.INTERNAL.), #40766=IfcRelSpaceBoundary('0yA0i411iHsCgcplgwOSGb',#18,'2ndLevel',$,#40710,$,#40765,.VIRTUAL.,.INTERNAL.), #40778=IfcRelSpaceBoundary('0ODNLoMsgheb15v30BerEW',#18,'2ndLevel',$,#40710,$,#40777,.VIRTUAL.,.INTERNAL.), #40791=IfcRelSpaceBoundary('0PH1Nt79QQEUa4oMlMVpey',#18,'2ndLevel',$,#40710,$,#40790,.VIRTUAL.,.INTERNAL.), #40810=IfcRelSpaceBoundary('2X9H9YvK6Uf7PiL7vHZLLY',#18,'2ndLevel',$,#40710,$,#40809,.VIRTUAL.,.INTERNAL.), #40883=IfcRelSpaceBoundary('2htvlj_19i7gXRGoW24gEa',#18,'2ndLevel',$,#40710,$,#40882,.VIRTUAL.,.INTERNAL.), #40894=IfcRelSpaceBoundary('28E4WFbEUrB6hfO$KgpwvV',#18,'2ndLevel',$,#40710,$,#40893,.VIRTUAL.,.INTERNAL.), #40904=IfcRelSp

## 2.3. Build graph

**RDF (Resource Description Framework)** represents data as triples: subject, predicate, and object. It’s commonly used in the Semantic Web to maintain a universal, flexible model of linked data. Relationships are defined via standardized vocabularies (e.g., RDF Schema, OWL), enabling robust data integration and inference across different domains.

**LPG (Labeled Property Graph)** organizes data into nodes (entities) and edges (relationships), both of which can have labels and properties. This approach is popular in many graph databases for direct, application-focused queries (often using query languages like Cypher). It’s simpler to model certain graph use cases without the semantic overhead of RDF vocabularies.

Our workflow is based on **Labeled Property Graph**

In [ ]:
G = nx.Graph()

# Add Rooms
for room in rooms:
    room_id = room.GlobalId
    G.add_node(room_id, **extract_properties(room), category="IfcSpace")

# Add Room-Element Connections
#    (Any building element bounding a room goes here with 'SURROUNDS'.)
for rel in space_boundaries:
    room = rel.RelatingSpace
    element = rel.RelatedBuildingElement
    if room and element:
        if element.GlobalId not in G.nodes:
            G.add_node(element.GlobalId, **extract_properties(element), category=element.is_a())
        # Connect the room to this element
        G.add_edge(room.GlobalId, element.GlobalId, relation="SURROUNDS")

# Add Wall-Window/Door Connections
#    (Direct 'VOIDS' edge from IfcWall* types to IfcDoor or IfcWindow.)
for rel in ifc_file.by_type("IfcRelVoidsElement"):
    wall = rel.RelatingBuildingElement
    opening = rel.RelatedOpeningElement

    for fill_rel in ifc_file.by_type("IfcRelFillsElement"):
        if fill_rel.RelatingOpeningElement == opening:
            filled_element = fill_rel.RelatedBuildingElement
            # Strict check: IfcWall*, IfcWallStandardCase, IfcCurtainWall --> [IfcDoor | IfcWindow]
            if (
                wall
                and filled_element
                and "Wall" in wall.is_a()
                and filled_element.is_a() in ["IfcDoor", "IfcWindow"]
            ):
                # Ensure both nodes (Wall + Door/Window) exist in the graph
                if wall.GlobalId not in G.nodes:
                    G.add_node(wall.GlobalId, **extract_properties(wall), category=wall.is_a())
                if filled_element.GlobalId not in G.nodes:
                    G.add_node(filled_element.GlobalId, **extract_properties(filled_element), category=filled_element.is_a())

                # Create a direct connection for the wall-door/window
                # print(f"VOIDS edge found: {wall.GlobalId} -> {filled_element.GlobalId}")
                G.add_edge(wall.GlobalId, filled_element.GlobalId, relation="VOIDS")


## 2.4. Generate 3D Layout and Visualization

In [ ]:
#Remove unconnected nodes
unconnected_nodes = [node for node, degree in G.degree() if degree == 0]
G.remove_nodes_from(unconnected_nodes)

# Recalculate layout (3D)
pos = nx.spring_layout(G, dim=3, seed=42)

# Extract updated node positions and colors
x_nodes, y_nodes, z_nodes = zip(*[pos[node] for node in G.nodes])
colors = [get_node_color(G.nodes[node].get("IfcType", "Undefined")) for node in G.nodes]

# Separate edge coordinates by relation type
voids_x, voids_y, voids_z = [], [], []
surrounds_x, surrounds_y, surrounds_z = [], [], []
other_x, other_y, other_z = [], [], []

for u, v, data in G.edges(data=True):
    x0, y0, z0 = pos[u]
    x1, y1, z1 = pos[v]

    # Add coordinates in Plotly "line segment" style: [x0, x1, None] so lines don't connect across edges
    if data.get("relation") == "VOIDS":
        voids_x.extend([x0, x1, None])
        voids_y.extend([y0, y1, None])
        voids_z.extend([z0, z1, None])
    elif data.get("relation") == "SURROUNDS":
        surrounds_x.extend([x0, x1, None])
        surrounds_y.extend([y0, y1, None])
        surrounds_z.extend([z0, z1, None])
    else:
        other_x.extend([x0, x1, None])
        other_y.extend([y0, y1, None])
        other_z.extend([z0, z1, None])

# Create separate edge traces
voids_trace = go.Scatter3d(
    x=voids_x, y=voids_y, z=voids_z,
    mode='lines',
    line=dict(width=2, color='red'),
    hoverinfo='none'
)

surrounds_trace = go.Scatter3d(
    x=surrounds_x, y=surrounds_y, z=surrounds_z,
    mode='lines',
    line=dict(width=2, color='gray'),
    opacity=0.5,
    hoverinfo='none'
)

other_trace = go.Scatter3d(
    x=other_x, y=other_y, z=other_z,
    mode='lines',
    line=dict(width=2, color='lightgray'),
    hoverinfo='none'
)

# Create the node trace
node_trace = go.Scatter3d(
    x=x_nodes, y=y_nodes, z=z_nodes,
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[
        f"{G.nodes[node].get('Name', 'N/A')} (" \
        f"{G.nodes[node].get('IfcType', 'Undefined')})"
        for node in G.nodes
    ],
    hoverinfo='text'
)

# Build figure
layout = go.Layout(
    title="3D IFC Wall-Door/Window Visualization",
    width=1200, height=800,
    scene=dict(xaxis=dict(title='X'),
               yaxis=dict(title='Y'),
               zaxis=dict(title='Z')),
    showlegend=False
)

fig = go.Figure(
    data=[voids_trace, surrounds_trace, other_trace, node_trace],
    layout=layout
)
fig.show()


In [ ]:
# SAVE THE PLOT AS HTML
fig.write_html("GML-02-NameLastname.html")


todo -> Location parameter!

## 2.5. Plot IfcWalls and IfcDoors/IfcWindows connections

In [ ]:
# Remove unconnected nodes
unconnected_nodes = [node for node, degree in G.degree() if degree == 0]
G.remove_nodes_from(unconnected_nodes)

# Recalculate layout (3D)
pos = nx.spring_layout(G, dim=3, seed=42)

# Extract node positions and colors (only for Walls, Doors, Windows)
wall_door_window_nodes = [
    node
    for node in G.nodes
    if G.nodes[node].get("IfcType") in [
        "IfcWall", "IfcWallStandardCase", "IfcCurtainWall", "IfcDoor", "IfcWindow"
    ]
]

x_nodes, y_nodes, z_nodes = zip(*[pos[node] for node in wall_door_window_nodes])
colors = [
    get_node_color(G.nodes[node].get("IfcType", "Undefined"))
    for node in wall_door_window_nodes
]

# Separate edge coordinates by relation type
voids_x, voids_y, voids_z = [], [], []
surrounds_x, surrounds_y, surrounds_z = [], [], []
other_x, other_y, other_z = [], [], []

for u, v, data in G.edges(data=True):
    x0, y0, z0 = pos[u]
    x1, y1, z1 = pos[v]

    # Add coordinates in Plotly "line segment" style: [x0, x1, None] so lines don't connect across edges
    if data.get("relation") == "VOIDS":
        voids_x.extend([x0, x1, None])
        voids_y.extend([y0, y1, None])
        voids_z.extend([z0, z1, None])
    elif data.get("relation") == "SURROUNDS":
        surrounds_x.extend([x0, x1, None])
        surrounds_y.extend([y0, y1, None])
        surrounds_z.extend([z0, z1, None])
    else:
        other_x.extend([x0, x1, None])
        other_y.extend([y0, y1, None])
        other_z.extend([z0, z1, None])

# Create separate edge traces
voids_trace = go.Scatter3d(
    x=voids_x, y=voids_y, z=voids_z,
    mode='lines',
    line=dict(width=2, color='red'),
    hoverinfo='none'
)

surrounds_trace = go.Scatter3d(
    x=surrounds_x, y=surrounds_y, z=surrounds_z,
    mode='lines',
    line=dict(width=2, color='gray'),
    opacity=0.05,
    hoverinfo='none'
)

other_trace = go.Scatter3d(
    x=other_x, y=other_y, z=other_z,
    mode='lines',
    line=dict(width=2, color='lightgray'),
    hoverinfo='none'
)

# Create the node trace
node_trace = go.Scatter3d(
    x=x_nodes, y=y_nodes, z=z_nodes,
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[
        f"{G.nodes[node].get('Name', 'N/A')} (" \
        f"{G.nodes[node].get('IfcType', 'Undefined')})"
        for node in wall_door_window_nodes
    ],
    hoverinfo='text'
)

#  Build figure
layout = go.Layout(
    title="3D IFC Wall-Door/Window Visualization",
    width=1200, height=800,
    scene=dict(xaxis=dict(title='X'),
               yaxis=dict(title='Y'),
               zaxis=dict(title='Z')),
    showlegend=False
)

fig = go.Figure(
    data=[voids_trace, surrounds_trace, other_trace, node_trace],
    layout=layout
)
fig.show()


## 2.6. Data check

Check and clean data to avoid duplicates!

In [ ]:
nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [ ]:
nodes

,GlobalId,Name,Description,ObjectType,IfcType,category
1GE3M13VHAWxhxwcp39Emh,1GE3M13VHAWxhxwcp39Emh,101,None,None,IfcSpace,IfcSpace
1GE3M13VHAWxhxwcp39EiJ,1GE3M13VHAWxhxwcp39EiJ,102,None,None,IfcSpace,IfcSpace
1GE3M13VHAWxhxwcp39Eiz,1GE3M13VHAWxhxwcp39Eiz,103,None,None,IfcSpace,IfcSpace
1GE3M13VHAWxhxwcp39Ei_,1GE3M13VHAWxhxwcp39Ei_,104,None,None,IfcSpace,IfcSpace
1GE3M13VHAWxhxwcp39Ei$,1GE3M13VHAWxhxwcp39Ei$,105,None,None,IfcSpace,IfcSpace
...,...,...,...,...,...,...
2UMN_OQKrAshZl3dHPuj5b,2UMN_OQKrAshZl3dHPuj5b,Basic Wall:Interior - 138mm Partition (1-hr):1...,None,Basic Wall:Interior - 138mm Partition (1-hr),IfcWallStandardCase,IfcWallStandardCase
2UMN_OQKrAshZl3dHPuj5a,2UMN_OQKrAshZl3dHPuj5a,Basic Wall:Interior - 138mm Partition (1-hr):1...,None,Basic Wall:Interior - 138mm Partition (1-hr),IfcWallStandardCase,IfcWallStandardCase
2UMN_OQKrAshZl3dHPuZuz,2UMN_OQKrAshZl3dHPuZuz,M_Single-Flush:0915 x 2134mm:165645,None,M_Single-Flush:0915 x 2134mm,IfcDoor,IfcDoor
2UMN_OQKrAshZl3dHPuj5c,2UMN_OQKrAshZl3dHPuj5c,Basic Wall:Interior - 138mm Partition (1-hr):1...,None,Basic Wall:Interior - 138mm Partition (1-hr),IfcWallStandardCase,IfcWallStandardCase


In [ ]:
# # Remove columns that contain a certain percentage of NaNs
# threshold = 0.5 * len(nodes)
# nodes = nodes.dropna(axis=1, thresh=threshold)  # Drop columns below the threshold

In [ ]:
nodes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 339 entries, 1GE3M13VHAWxhxwcp39Emh to 2UMN_OQKrAshZl3dHPuj5g
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   GlobalId     339 non-null    object
 1   Name         339 non-null    object
 2   Description  0 non-null      object
 3   ObjectType   248 non-null    object
 4   IfcType      339 non-null    object
 5   category     339 non-null    object
dtypes: object(6)
memory usage: 26.6+ KB


In [ ]:
nodes.head(3)

,GlobalId,Name,Description,ObjectType,IfcType,category
1GE3M13VHAWxhxwcp39Emh,1GE3M13VHAWxhxwcp39Emh,101,None,None,IfcSpace,IfcSpace
1GE3M13VHAWxhxwcp39EiJ,1GE3M13VHAWxhxwcp39EiJ,102,None,None,IfcSpace,IfcSpace
1GE3M13VHAWxhxwcp39Eiz,1GE3M13VHAWxhxwcp39Eiz,103,None,None,IfcSpace,IfcSpace


In [ ]:
# Replace Nans with a string 'N/A'
nodes.fillna('N/A', inplace=True)

In [ ]:
nodes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 339 entries, 1GE3M13VHAWxhxwcp39Emh to 2UMN_OQKrAshZl3dHPuj5g
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   GlobalId     339 non-null    object
 1   Name         339 non-null    object
 2   Description  339 non-null    object
 3   ObjectType   339 non-null    object
 4   IfcType      339 non-null    object
 5   category     339 non-null    object
dtypes: object(6)
memory usage: 26.6+ KB


In [ ]:
nodes['IfcType'].value_counts()

,count
IfcType,
IfcWallStandardCase,117
IfcSpace,91
IfcDoor,84
IfcWindow,24
IfcColumn,11
IfcCurtainWall,5
IfcWall,4
IfcSlab,3


In [ ]:
nodes['category'].value_counts()

,count
category,
IfcWallStandardCase,117
IfcSpace,91
IfcDoor,84
IfcWindow,24
IfcColumn,11
IfcCurtainWall,5
IfcWall,4
IfcSlab,3


In [ ]:
edges = pd.DataFrame(G.edges(data=True), columns=['source', 'target', 'attributes'])

In [ ]:
edges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862 entries, 0 to 861
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   source      862 non-null    object
 1   target      862 non-null    object
 2   attributes  862 non-null    object
dtypes: object(3)
memory usage: 20.3+ KB


In [ ]:
edges.head()

,source,target,attributes
0,1GE3M13VHAWxhxwcp39Emh,3FZFp0nq9AAhRAecLpGfvv,{'relation': 'SURROUNDS'}
1,1GE3M13VHAWxhxwcp39Emh,3a58fQxqD2numVo$eTOgS$,{'relation': 'SURROUNDS'}
2,1GE3M13VHAWxhxwcp39Emh,2Tt7$hRrHBKvoaUIFPAz_N,{'relation': 'SURROUNDS'}
3,1GE3M13VHAWxhxwcp39Emh,3n9ek_acHEfRCBQHch6_cD,{'relation': 'SURROUNDS'}
4,1GE3M13VHAWxhxwcp39EiJ,2Tt7$hRrHBKvoaUIFPAz_N,{'relation': 'SURROUNDS'}


In [ ]:
# Check for edges types
edges['relation_type'] = edges['attributes'].apply(lambda x: x.get('relation', None))
unique_relations = edges['relation_type'].unique()
print(unique_relations)


['SURROUNDS' 'VOIDS']


## 2.7. Save `csv` files for nodes and edges

In [ ]:
# nodes.to_csv('/content/drive/MyDrive/IFC/edges_01.csv')
# edges.to_csv('/content/drive/MyDrive/IFC/nodes_01.csv')
nodes.to_json('/content/edges_01.json')
edges.to_json('/content/nodes_01.json')

## BONUS: visualizg in the web

In [ ]:
!pip install ipysigma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.1 MB/s eta 0:00:00


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# Visualize
from ipysigma import Sigma
sigma  = Sigma(G)

# Display the widget in the notebook
display(sigma)

Sigma(nx.Graph with 339 nodes and 862 edges)

In [ ]:
# Export the graph to an HTML file
sigma.write_html(
    G,
    './BDG_IFC.html',
    fullscreen=True,
    node_color= 'dept',  # Use the 'color' attribute for course type
    node_size_range=(1, 15),
    default_edge_type='line',
    node_border_color_from='node',
    default_node_label_size=12,
    edge_size_range=(1, 15),
    node_size=lambda node: G.degree(node)
)